## Pipiline para agrupamentos de Texto

* Converter os documentos de entrada em embeddings com um modelo de embedding.
* Reduzir a dimensionalidade dos embeddings com um modelo de redução de dimensionalidade.
* Encontrar grupos de documentos semanticamente similares com um modelo de cluster.

# Preparação dos dados

In [155]:
import json

file_path = "infomoney_noticias_limpas.json"

# Carregar dados do arquivo JSON
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

contents = []

for article in data["articles"]:
    if "conteudo" in article:
        contents.append(article["conteudo"])

In [156]:
contents

['O multimilionário Elon Musk, antigo aliado do presidente americano, Donald Trump, cujo governo ele abandonou recentemente anunciou neste sábado, 5, a criação de seu próprio movimento político, o Partido da América.\nDecepcionado com a lei orçamentária aprovada semana passada, que elevará a dívida nacional, Musk havia prometido nos últimos dias criar sua própria legenda. “Quando se trata de arruinar nosso país por meio do desperdício e da corrupção, vivemos em um sistema de partido único, não em uma democracia”, disse Musk.\nO presidencialismo americano é escorado no bipartidarismo – republicanos e democratas. Uma terceira força poderia “roubar” votos das duas legendas e bagunçar as eleições.',
 'O presidente Donald Trump revelou nesta quinta-feira (3) sua intenção de organizar uma luta de UFC no gramado da Casa Branca para celebrar os 250 anos da independência dos Estados Unidos, que será comemorada em 4 de julho de 2026. O evento, que poderia reunir entre 20 mil e 25 mil espectadore

- Para a representação vetorial dos textos, foi selecionado um modelo de embedding específico para a língua portuguesa, o BERTimbau.

Como segue abaixo:

In [157]:
from sentence_transformers import SentenceTransformer

# Criar um embedding para cada resumo
embedding_model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")
embeddings = embedding_model.encode(contents, show_progress_bar=True)

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.
Batches: 100%|██████████| 2/2 [00:01<00:00,  1.19it/s]


In [158]:
embeddings.shape

(40, 768)

* Alta dimensionalidade

In [159]:
print(embeddings)

[[-0.20318638 -0.09996612  0.45102838 ... -0.1733559   0.07085326
  -0.28037894]
 [-0.16795228 -0.04368041  0.65698785 ...  0.03931916 -0.09039972
  -0.39492   ]
 [-0.27284724 -0.09413554  0.49340805 ...  0.18009481  0.01982507
  -0.2832254 ]
 ...
 [-0.3489291  -0.03460573  0.5120177  ...  0.28621197  0.07401264
  -0.15650555]
 [-0.27382964 -0.06616925  0.65771115 ...  0.00806899  0.06353793
  -0.3349498 ]
 [-0.10400479 -0.13638213  0.4811678  ...  0.2662266  -0.05356794
  -0.35427392]]


# Redução da Dimensionalidade dos Embeddings

> Optou-se por utilizar o método Uniform Manifold Approximation and Projection (UMAP), dada sua maior capacidade de lidar com relacionamentos e estruturas não lineares em comparação ao PCA

In [ ]:
from umap import UMAP

# Reduzimos os embeddings de entrada de 768 dimensões para 30 dimensões
umap_model = UMAP(
    n_components=30 , min_dist=0.02, metric='cosine',
    random_state=42
)
reduced_embeddings = umap_model.fit_transform(embeddings)

## Agrupamento dos Embeddings Reduzidos

* A etapa final do agrupamento é encontrar grupos de documentos semanticamente semelhantes.

* Em vez de algoritmos baseados em centróides como o k-means, que exigem o número de clusters antecipadamente, o livro sugere um algoritmo baseado em densidade como o HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise).

* HDBSCAN calcula livremente o número de clusters e pode detectar outliers (pontos de dados que não pertencem a nenhum cluster), que são ignorados e não forçados a pertencer a um cluster. Isso é útil para dados com papers de nicho, por exemplo.

* Parâmetros como min_cluster_size (tamanho mínimo para um cluster) e metric ("euclidean") são definidos para o HDBSCAN. Reduzir o min_cluster_size resulta em mais clusters.

* Os outliers são designados com o rótulo -1

In [136]:
from hdbscan import HDBSCAN

# Ajustamos o modelo e extraímos os clusters
hdbscan_model = HDBSCAN(
    min_cluster_size=2, metric="euclidean",
    cluster_selection_method="eom"
).fit(reduced_embeddings)

clusters = hdbscan_model.labels_

# Quantos clusters geramos?
len(set(clusters))

8

In [143]:
import numpy as np

# Imprimir os primeiros três documentos no cluster 0
cluster = 0
for index in np.where(clusters==cluster)[0][:3]:
    print(contents[index][:300] + "... \n")

O multimilionário Elon Musk, antigo aliado do presidente americano, Donald Trump, cujo governo ele abandonou recentemente anunciou neste sábado, 5, a criação de seu próprio movimento político, o Partido da América.
Decepcionado com a lei orçamentária aprovada semana passada, que elevará a dívida nac... 

O podcast Sports Pundit recebeu na semana passada Rob Pilgrim, chefe de esportes e horário nobre do YouTube EMEA. Durante 45 minutos, o executivo centrou a conversa na falta de compreensão sobre como a plataforma está reposicionando o valor do esporte premium. Não para sabotar as emissoras tradicion... 

O apresentador Luciano Huck criticou o sistema político do Brasil neste sábado, 5, no seu perfil do X (antigo Twitter). Resgatando um trecho da participação do ex-presidente Fernando Henrique Cardoso no programa Roda Viva em 1993, onde critica o Congresso Nacional, Huck afirmou que o “presidencialis... 



In [144]:
from bertopic import BERTopic

# Treinar nosso modelo com nossos modelos previamente definidos
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
).fit(contents, embeddings)

2025-07-06 13:11:39,194 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-06 13:11:39,278 - BERTopic - Dimensionality - Completed ✓
2025-07-06 13:11:39,279 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-06 13:11:39,282 - BERTopic - Cluster - Completed ✓
2025-07-06 13:11:39,284 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-06 13:11:39,311 - BERTopic - Representation - Completed ✓


In [145]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8,-1_de_do_em_que,"[de, do, em, que, para, da, com, suzy, por, fa...",[Após enfrentar a maior queda anual de suas aç...
1,0,8,0_de_que_os_em,"[de, que, os, em, uma, as, para, do, um, mais]",[É natural que superfãs queiram ter um pedaço ...
2,1,6,1_de_com_do_em,"[de, com, do, em, pontos, no, da, que, acima, ...","[Na quinta-feira (03), o dólar à vista recuou ..."
3,2,6,2_de_do_que_da,"[de, do, que, da, em, no, não, para, com, na]","[A Expert XP, maior festival de investimentos ..."
4,3,5,3_de_em_no_na,"[de, em, no, na, do, que, da, os, com, foi]",[Restam somente quatro equipes no Mundial de C...
5,4,3,4_de_vibra_da_natura,"[de, vibra, da, natura, que, inpasa, para, inv...",[SÃO PAULO (Reuters) – O Nubank (BDR:ROXO34) i...
6,5,2,5_de_renda_um_do,"[de, renda, um, do, em, ano, 100, retorno, com...",[O fundo imobiliário HSI Renda Urbana (HSRE11)...
7,6,2,6_fundos_investidor_de_que,"[fundos, investidor, de, que, para, ao, dos, o...",[Acumular R$ 50 mil não é uma tarefa fácil par...


* O BERTopic primeiro cria clusters de documentos semanticamente semelhantes, utilizando embeddings de modelos Transformer. Para a representação dos tópicos, ele emprega uma abordagem de 'bolsa de palavras' (bag-of-words) aprimorada por c-TF-IDF, o que permite rotular os clusters com palavras-chave mais significativas, ponderando a sua relevância para o tópico em vez de apenas a frequência geral. 

De fato, é fundamental e possível melhorar essas representações posteriormente com o uso de diversos modelos de representação (também chamados de rerankers), aproveitando a modularidade do framework

In [146]:
# Visualizar tópicos e documentos
fig = topic_model.visualize_documents(
    contents,
    reduced_embeddings=reduced_embeddings,
    width=1200,
    hide_annotations=True
)

# Atualizar fontes da legenda para visualização mais fácil
fig.update_layout(font=dict(size=16))

In [148]:
# Visualizar gráfico de barras com palavras-chave ranqueadas
topic_model.visualize_barchart()

# Visualizar relacionamentos entre tópicos
topic_model.visualize_heatmap(n_clusters=6)

# Visualizar a estrutura hierárquica potencial dos tópicos
topic_model.visualize_hierarchy()

# Adicionando modelos de representação

In [149]:
# Salvar representações originais
from copy import deepcopy
original_topics = deepcopy(topic_model.topic_representations_)

In [150]:
import pandas as pd

def topic_differences(model, original_topics, nr_topics=5):
    """Mostrar as diferenças nas representações de tópicos entre dois modelos"""
    df = pd.DataFrame(columns=["Topic", "Original", "Updated"])
    for topic in range(nr_topics):
        # Extrair as 5 principais palavras por tópico por modelo
        og_words = " | ".join(list(zip(*original_topics[topic]))[0][:5])
        new_words = " | ".join(list(zip(*model.get_topic(topic)))[0][:5])
        df.loc[len(df)] = [topic, og_words, new_words]
    
    return df

In [151]:
from bertopic.representation import KeyBERTInspired

# Atualizar representações de tópicos usando KeyBERTInspired
representation_model = KeyBERTInspired()
topic_model.update_topics(contents, representation_model=representation_model)

# Mostrar diferenças de tópicos
topic_differences(topic_model, original_topics)

,Topic,Original,Updated
0,0,de | que | os | em | uma,traders | mystic | texas | eua | amber
1,1,de | com | do | em | pontos,ibovespa | shoppings | em142 | em143 | oifr
2,2,de | do | que | da | em,cazétv | pilgrim | tv | paulo | fhc
3,3,de | em | no | na | do,fluminense | chelsea | ufrj | paulo | guarulhos
4,4,de | vibra | da | natura | que,societário | etanol | reuters | bdr | chairman


Para o refinamento dos tópicos, foi utilizado o modelo KeyBERTInspired, que melhorou significativamente a coerência e a interpretabilidade das representações. 

O funcionamento do KeyBERTInspired é um processo de refinamento que começa com as palavras-chave já identificadas pelo c-TF-IDF. Primeiramente, o modelo trata todos os documentos de um tópico como um texto único e contínuo, gerando um vetor de embedding que representa o significado semântico central do tópico como um todo. Paralelamente, ele cria um vetor de embedding para cada uma dessas palavras candidatas. A etapa seguinte é a comparação, onde o modelo utiliza a similaridade de cosseno para medir a distância semântica entre o vetor do tópico e o vetor de cada palavra. As palavras cujos vetores se mostram mais próximos ao vetor geral do tópico são, então, selecionadas e ranqueadas para formar a nova e mais coerente representação.

No entanto, foi identificada uma limitação em seu comportamento: o modelo tende a ignorar ou sub-representar termos específicos como siglas.

In [152]:
from bertopic.representation import MaximalMarginalRelevance

# Atualizar nossas representações de tópicos para MaximalMarginalRelevance
representation_model = MaximalMarginalRelevance(diversity=0.2)
topic_model.update_topics(contents, representation_model=representation_model)

# Mostrar diferenças de tópicos
topic_differences(topic_model, original_topics)

,Topic,Original,Updated
0,0,de | que | os | em | uma,que | os | para | do | não
1,1,de | com | do | em | pontos,do | que | períodos | para | alta
2,2,de | do | que | da | em,do | não | para | uma | xp
3,3,de | em | no | na | do,do | da | os | polícia | apreendidos
4,4,de | vibra | da | natura | que,vibra | natura | inpasa | feira | empresa


O método Maximal Marginal Relevance (MMR) é utilizado no BERTopic como um modelo de representação (ou reranker) para diversificar as representações dos tópicos que foram inicialmente geradas por c-TF-IDF. O BERTopic, sendo um framework modular, permite a integração desses "blocos Lego especiais" para aprimorar as representações de tópico

Mecanismo de Seleção:

1. O algoritmo começa com um conjunto inicial de palavras-chave candidatas (por exemplo, 30 palavras-chave).

2. Ele calcula os embeddings dessas palavras-chave candidatas.

3. Em seguida, ele seleciona iterativamente a próxima melhor palavra-chave a ser adicionada ao conjunto final, considerando tanto sua relevância para o tópico quanto sua diversidade em relação às palavras-chave já selecionadas.

4. Isso significa que o MMR filtra palavras redundantes e mantém apenas aquelas que contribuem com algo novo para a representação do tópico